In [1]:
# imports
import pandas as pd
import scipy as sp 
import math
import plotly.express as px
import numpy as np

from _mic_position_functions import calculate_distance, parametric_model, calc_error_optimized_coords

steps:
* import meaasured 3d mic positions and distance measurements
* only use points with measured distance
* calculate sum of squarred errors
* build a mic positions model with parameters (offsets and tilt)
* optimize those paramters for minimal sum of squarred errors
* return optimized paramters and mic positions

In [2]:
# read meaasured 3d mic positions
df_mic_coords = pd.read_excel("data/3d_measured_coords.xlsx", index_col=0)

# read distance measurements
df_distances = pd.read_excel("data/mic_pos_distances.xlsx")

In [3]:
# check if calculated distances match

print(df_distances.head())

print("")
for index, row in df_distances.iterrows():
    
    dist = calculate_distance(df_mic_coords, row[0], row[1], print_dist=False)

    if math.isclose(dist, df_distances["dist calculated"][index], abs_tol=0.001) == False:
        print(index)
        print("different distance")

print("all distances checked")

  Mic_Index 1 Mic_Index 2  dist measured  dist calculated  error
0         D13         D16          1.262            1.256  0.006
1         B17         C16          1.570            1.549  0.021
2         C10         D10          1.144            1.126  0.018
3          C2         B15          1.577            1.579 -0.002
4          A7          B6          1.445            1.422  0.023

18
different distance
all distances checked


In [4]:
# get all mic positions used for measurng distances (no further used)
# get indixes
measured_mic_pos = []

for mic_index_temp in df_distances["Mic_Index 1"]:
    measured_mic_pos.append(mic_index_temp)

for mic_index_temp in df_distances["Mic_Index 2"]:
    measured_mic_pos.append(mic_index_temp)

measured_mic_pos = set(measured_mic_pos) # remove double values
measured_mic_pos = sorted(measured_mic_pos) # sort values

# filter dataframe
df_mic_coords_filtered = df_mic_coords[df_mic_coords.isin(measured_mic_pos).any(axis=1)]


print(len(measured_mic_pos), "different mics used for measurng distances")
print("")
print(df_mic_coords_filtered)

37 different mics used for measurng distances

   Mic_Index  Plane      X      Y      Z
2         B3  right  1.370  1.212  0.287
3         B4  right  1.370  0.150  0.380
5         B6  right  1.370  0.559  0.629
7         B8  right  1.370  0.233  0.718
11       B12  right  1.370  0.739  1.194
12       B13  right  1.370  1.391  1.257
13       B14  right  1.370  1.012  1.372
14       B15  right  1.370  0.504  1.485
15       B16  right  1.370  0.156  1.510
16       B17  right  1.370  0.874  1.689
18        C2  front  0.485  0.028  0.267
20        C4  front  0.076  0.028  0.380
21        C5  front  0.827  0.028  0.425
26       C10  front  1.099  0.028  0.990
28       C12  front  0.667  0.028  1.194
32       C16  front  0.085  0.028  1.508
33       C17  front  0.799  0.028  1.691
34        D1   left  0.028  0.737  0.023
36        D3   left  0.028  0.339  0.289
37        D4   left  0.028  1.400  0.385
38        D5   left  0.028  0.649  0.427
40        D7   left  0.028  0.223  0.677
43       D

In [5]:
# calculate sum of squarred errors

for index, row in df_distances.iterrows():
    
    dist = calculate_distance(df_mic_coords, row[0], row[1], print_dist=False)
    sq_error = (dist - df_distances["dist measured"][index])**2
    df_distances.loc[index, "squared error"] = sq_error

sum_sq_errors_default = df_distances["squared error"].sum()
print(round(sum_sq_errors_default, 5), "= default sum of squared errors")


0.0261 = default sum of squared errors


In [6]:
# Parameter

# translations = shift = offset

# Rotations:
# Roll (rotation around the plane's local X-axis)
# Pitch (rotation around the plane's local Y-axis)
# Yaw (rotation around the plane's local Z-axis) --> assumption: no yaw

# shift is in m
# roll & pitch in degrees

# top plane
A_x_shift = 0
A_y_shift = 0

# left plane
# B_x_shift = 0 # keep 0 for reference
B_y_shift = 0
B_z_shift = 0
B_roll = 0 # rotation over 3d y axis 
B_pitch = 0 # rotation over 3d z axis

# front plane
C_x_shift = 0
# C_y_shift = 0 # keep 0 for reference
C_z_shift = 0
C_roll = 0 # rotation over 3d x axis 
C_pitch = 0 # rotation over 3d z axis

# right_plane
D_x_shift = 0
D_y_shift = 0
D_z_shift = 0
D_roll = 0 # rotation over 3d y axis 
D_pitch = 0 # rotation over 3d z axis


initial_parameters = [
    A_x_shift,
    A_y_shift,
    B_y_shift,
    B_z_shift,
    B_roll,
    B_pitch,
    C_x_shift,
    C_z_shift,
    C_roll,
    C_pitch,
    D_x_shift,
    D_y_shift,
    D_z_shift,
    D_roll,
    D_pitch
    ]


print(len(initial_parameters), "parameter in total")
print(initial_parameters)

15 parameter in total
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [7]:

def optimizer_wrapper(initial_parameters, df_mic_coords=df_mic_coords, df_distances=df_distances):
    df_mic_coords_moved = parametric_model(df_mic_coords, initial_parameters)
    sum_sq_errors, _ = calc_error_optimized_coords(df_distances, df_mic_coords_moved)

    return sum_sq_errors

In [8]:
result = sp.optimize.minimize(optimizer_wrapper, initial_parameters)

print("Optimal variables:", result.x)
print("Optimal function value:", result.fun)

Optimal variables: [-0.04069949 -0.03433035 -0.03710266 -0.0127838  -0.85888897 -0.85392481
  0.00751255  0.00916716  1.71427966 -1.30208812  0.05220112 -0.02361945
 -0.02754176 -1.51923372 -0.72119733]
Optimal function value: 0.001669140461021823


In [9]:
df_mic_coords_optimized = parametric_model(df_mic_coords, result.x)

# Create the first 3D scatter plot
fig = px.scatter_3d(df_mic_coords, 
                    x='X', 
                    y='Y', 
                    z='Z', 
                    text='Mic_Index', 
                    title='3D Scatter Plot', 
                    color='Plane', 
                    width=1000, 
                    height=1000)

# Create the second 3D scatter plot (optimized data)
fig_optimized = px.scatter_3d(df_mic_coords_optimized, 
                               x='X_optimized', 
                               y='Y_optimized', 
                               z='Z_optimized', 
                               text='Mic_Index', 
                               color='Plane')

# Customize the color of the second plot to differentiate it
for trace in fig_optimized.data:
    trace.marker.color = 'rgba(0, 0, 0, 0.7)'  # black with some transparency
    trace.name = 'Optimized Data'  # Label in the legend

# Add each trace from the second figure to the first
for trace in fig_optimized.data:
    fig.add_trace(trace)



# Display the combined figure
fig.update_layout(scene=dict(
    xaxis=dict(range=[1.5, 0]), # x-Achse umdrehen - "linke hand regel"
))
fig.show()


In [11]:
# save 3d coords as xml (for acoular)

# acoular needs rows in order from patch plan (equals channels in acoular)
df_mic_coords_patch_plan = df_mic_coords_optimized.copy()
df_patch_plan = pd.read_excel("data/patch_plan.xlsx")
df_mic_coords_patch_plan = df_mic_coords_patch_plan.merge(df_patch_plan, on="Mic_Index")

df_mic_coords_patch_plan.rename(columns={"Plane_x": "Plane"}, inplace=True)
df_mic_coords_patch_plan = df_mic_coords_patch_plan.drop(columns="Plane_y")
df_mic_coords_patch_plan.sort_values(by=["Array_Channel"], inplace=True)



In [12]:

def df2xml(df, x, y, z, name):
    coordinates = df[[x, y, z]].to_numpy()
    coordinates = np.round(coordinates, 3)

    headerXML = f'''<?xml version="1.0" encoding="utf-8"?> \n<MicArray name="bassoon_cage_64_{name}">\n'''
    elements = ""
    for i, element in enumerate(coordinates):
        elements = elements + f'  <pos Name="Point {i}" x="{element[0]}" y="{element[1]}" z="{element[2]}"/>\n'

    footerXML = '''
    </MicArray>
    '''

    XML = (headerXML+elements+footerXML)
    outFile = open(f"data/bassoon_cage_64_{name}.xml","w")
    outFile.write(XML)
    outFile.close()

# write default and optimized 3d coords to xml
df2xml(df_mic_coords_patch_plan, "X", "Y", "Z", "calculated_default")
df2xml(df_mic_coords_patch_plan, "X_optimized", "Y_optimized", "Z_optimized", "optimized")

In [13]:
# save optimizer results to file
df_mic_coords_patch_plan.to_excel("data/mic_coords_final.xlsx")

# open file in write mode
with open("data/optimized_parameters.txt", 'w') as fp:
    for item in result.x:
        # write each item on a new line
        fp.write("%s\n" % item)